In [1]:
# Version Genie de Cosine del dia 09/06/2025 en la que no se eliminan las trayectorias de los objetos que desaparecen.

import cv2
from ultralytics import YOLO

# 1. Cargar el modelo YOLOv11 preentrenado
model = YOLO('yolo11n.pt')

# 2. Leer el video de entrada
path = 'C:/Users/gtoma/Master_AI_Aplicada/UEM_Master_AI_07042025/UEM_Trabajo/Sesiones_UEM/Manuel_Garcia_VISION/Actividad_3_YOLOV11/'
video_path = path + 'comma_small.mp4'
cap = cv2.VideoCapture(video_path)

# 3. Preparar el writer para el video de salida
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(path + 'video_con_detecciones_trazasforever.mp4', fourcc, int(cap.get(cv2.CAP_PROP_FPS)),
                    (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# 4. Diccionario para guardar trayectorias por ID
trajectories = dict()

# 5. Procesar frame a frame
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv11 inferencia con tracking activado
    my_tracker = 'bytetrack.yml'  # Ruta al modelo de tracking preentrenado
    results = model.track(frame, persist=True, tracker=my_tracker)  # Usa tracking basado en ByteTrack

    # Filtrar solo los vehículos y las personas (revisa clases: car, bus, truck, etc.)
    for det in results:
        if det.boxes is not None:
            for box in det.boxes:
                cls_id = int(box.cls[0])
                track_id = int(box.id[0]) if box.id is not None else None
                # Ajusta según las clases de vehículos en YOLOv11
                # 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench',
                # 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
                # 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
                # 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant',
                # 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 
                # 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
                if model.names[cls_id] in ['person', 'bycicle', 'car', 'motorcycle', 'bus', 'truck', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench']:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    # Dibujar bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
                    cv2.putText(frame, f"{model.names[cls_id]} ID:{track_id}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

                    # Guardar centro del bounding box para la trayectoria
                    cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
                    if track_id not in trajectories:
                        trajectories[track_id] = []
                    trajectories[track_id].append((cx, cy))

    # Dibujar trazas de cada objeto
    for track_id, points in trajectories.items():
        for i in range(1, len(points)):
            cv2.line(frame, points[i-1], points[i], (255,0,0), 2)

    # Escribir frame al video de salida
    out.write(frame)
    
    # --- MOSTRAR EL FRAME EN VENTANA ---
    cv2.imshow("Tracking en tiempo real", frame)
    # Salir si se pulsa 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()
print("¡Video generado con bounding boxes y trayectorias guardado como 'video_con_detecciones.mp4'!")


0: 384x640 1 traffic light, 64.7ms
Speed: 2.4ms preprocess, 64.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 56.6ms
Speed: 2.4ms preprocess, 56.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 traffic light, 52.2ms
Speed: 1.2ms preprocess, 52.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 48.6ms
Speed: 1.2ms preprocess, 48.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 traffic light, 47.9ms
Speed: 1.0ms preprocess, 47.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 48.3ms
Speed: 1.0ms preprocess, 48.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 315.5ms
Speed: 2.5ms preprocess, 315.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 307.6ms
Speed: 4.4ms preprocess, 307.6

In [6]:
# Version Genie de Cosine del dia 09/06/2025 en la que si se eliminan las trayectorias de los objetos que desaparecen.

import cv2
from ultralytics import YOLO

# 1. Cargar el modelo YOLOv11 preentrenado
model = YOLO('yolo11n.pt')

# 2. Leer el video de entrada
path = 'C:/Users/gtoma/Master_AI_Aplicada/UEM_Master_AI_07042025/UEM_Trabajo/Sesiones_UEM/Manuel_Garcia_VISION/Actividad_3_YOLOV11/'
video_path = path + 'comma_small.mp4'
cap = cv2.VideoCapture(video_path)

# 3. Preparar el writer para el video de salida
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(path + 'video_con_detecciones_sintrazasperyptorojo.mp4', fourcc, int(cap.get(cv2.CAP_PROP_FPS)),
                    (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# 4. Diccionario para guardar trayectorias por ID
trajectories = dict()

# 5. Procesar el video frame por frame
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv11 inferencia con tracking activado
    my_tracker = 'bytetrack.yml'  # Ruta al modelo de tracking preentrenado
    results = model.track(frame, persist=True, tracker=my_tracker)  # Usa tracking basado en ByteTrack

    active_ids = set()  # IDs presentes en este frame
    
    # Filtrar solo los vehículos y las personas (revisa clases: car, bus, truck, etc.)
    for det in results:
        if det.boxes is not None:
            for box in det.boxes:
                cls_id = int(box.cls[0])
                track_id = int(box.id[0]) if box.id is not None else None
                # Ajusta según las clases de vehículos en YOLOv11
                # 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench',
                # 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
                # 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
                # 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant',
                # 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 
                # 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
                if model.names[cls_id] in ['person', 'bycicle', 'car', 'motorcycle', 'bus', 'truck', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench']:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    # Dibujar bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
                    cv2.putText(frame, f"{model.names[cls_id]} ID:{track_id}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
                    # Guardar centro del bounding box para la trayectoria
                    cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
                    cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)  # <-- punto rojo
                    if track_id not in trajectories:
                        trajectories[track_id] = []
                    trajectories[track_id].append((cx, cy))
                    active_ids.add(track_id)

    # --- Eliminar trayectorias de IDs que ya no están activos ---
    to_remove = [tid for tid in trajectories if tid not in active_ids]
    for tid in to_remove:
        del trajectories[tid]

    # Dibujar trazas de objetos actualmente visibles
    for track_id, points in trajectories.items():
        for i in range(1, len(points)):
            cv2.line(frame, points[i-1], points[i], (255,0,0), 2)

    # Escribir frame al video de salida
    out.write(frame)
    # Mostrar el frame con detecciones y trayectorias
    cv2.imshow("Tracking en tiempo real", frame)
    # Esperar por la tecla 'q' para salir
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()

print("¡Video generado con trayectorias que desaparecen al perder el objeto!")


0: 384x640 1 traffic light, 72.6ms
Speed: 1.6ms preprocess, 72.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 54.9ms
Speed: 1.1ms preprocess, 54.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 traffic light, 53.7ms
Speed: 1.1ms preprocess, 53.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 50.6ms
Speed: 1.0ms preprocess, 50.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 traffic light, 50.4ms
Speed: 0.9ms preprocess, 50.4ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 51.2ms
Speed: 1.3ms preprocess, 51.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 46.1ms
Speed: 1.6ms preprocess, 46.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 44.9ms
Speed: 1.0ms preprocess, 44.9ms i

Notas importantes:
Cambia ['car', 'truck', 'bus', 'motorcycle'] según las clases que YOLOv11 tenga para vehículos (puedes imprimir model.names para verlas).
El tracking usa ByteTrack (incluido en Ultralytics). Puedes cambiar el tracker si lo deseas (ver docs Ultralytics).
Este código guarda el video con bbboxes y trayectorias en el mismo tamaño y fps que el original.

In [ ]:
print(model.names)  # Imprime las clases del modelo YOLOv11

In [ ]:
import ultralytics
print(ultralytics.__file__) # Indica donde estan instalado el paquete ultralytics
print(ultralytics.__version__) # Indica la version del paquete ultralytics

In [3]:
# Version Genie de Cosine del dia 09/06/2025 con trazas fade-away
import cv2
from ultralytics import YOLO

# 1. Cargar el modelo YOLOv11 preentrenado
model = YOLO('yolo11n.pt')

# 2. Leer el video de entrada
path = 'C:/Users/gtoma/Master_AI_Aplicada/UEM_Master_AI_07042025/UEM_Trabajo/Sesiones_UEM/Manuel_Garcia_VISION/Actividad_3_YOLOV11/'
video_path = path + 'comma_small.mp4'
cap = cv2.VideoCapture(video_path)

# 3. Preparar el writer para el video de salida
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(path + 'video_con_detecciones_fadeaway.mp4', fourcc, int(cap.get(cv2.CAP_PROP_FPS)),
                    (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# 4. Diccionario para guardar trayectorias por ID
trajectories = dict()           # track_id: list of points
fade_outs = dict()              # track_id: frames restantes para desvanecer
FADE_FRAMES = 20                # Cuantos frames dura el desvanecimiento

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(frame, persist=True, tracker="bytetrack.yaml")
    active_ids = set()

    for det in results:
        if det.boxes is not None:
            for box in det.boxes:
                cls_id = int(box.cls[0])
                track_id = int(box.id[0]) if box.id is not None else None
                # Ajusta según las clases de vehículos en YOLOv11
                # 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench',
                # 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
                # 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
                # 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant',
                # 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 
                # 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
                if model.names[cls_id] in ['person', 'bycicle', 'car', 'motorcycle', 'bus', 'truck', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench']:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
                    cv2.putText(frame, f"{model.names[cls_id]} ID:{track_id}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
                    cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
                    if track_id not in trajectories:
                        trajectories[track_id] = []
                    trajectories[track_id].append((cx, cy))
                    active_ids.add(track_id)
                    # Si un objeto vuelve a aparecer, elimina su fade_out (si lo tenía)
                    if track_id in fade_outs:
                        del fade_outs[track_id]

    # Trayectorias de objetos que desaparecieron: inicia fade
    for tid in list(trajectories.keys()):
        if tid not in active_ids and tid not in fade_outs:
            fade_outs[tid] = FADE_FRAMES

    # Dibujar trayectorias activas (azul fuerte)
    for track_id in active_ids:
        pts = trajectories[track_id]
        for i in range(1, len(pts)):
            cv2.line(frame, pts[i-1], pts[i], (255,0,0), 2)

    # Dibujar trayectorias en desvanecimiento
    to_remove = []
    for track_id, fade in fade_outs.items():
        pts = trajectories[track_id]
        alpha = fade / FADE_FRAMES
        color = (int(255*alpha), int(255*alpha), 255)  # azul muy claro → blanco
        for i in range(1, len(pts)):
            cv2.line(frame, pts[i-1], pts[i], color, 2)
        fade_outs[track_id] -= 1
        if fade_outs[track_id] <= 0:
            to_remove.append(track_id)

    # Eliminar trayectorias que ya se desvanecieron
    for tid in to_remove:
        del trajectories[tid]
        del fade_outs[tid]

    out.write(frame)
    cv2.imshow("Tracking en tiempo real", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("¡Video generado con trayectorias que se desvanecen suavemente al perder el objeto!")


0: 384x640 1 traffic light, 52.5ms
Speed: 1.5ms preprocess, 52.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 56.0ms
Speed: 1.0ms preprocess, 56.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 traffic light, 57.9ms
Speed: 1.3ms preprocess, 57.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 53.3ms
Speed: 1.3ms preprocess, 53.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 traffic light, 51.2ms
Speed: 1.3ms preprocess, 51.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 48.6ms
Speed: 1.0ms preprocess, 48.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 47.1ms
Speed: 1.0ms preprocess, 47.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 44.3ms
Speed: 1.1ms preprocess, 44.3ms i